# 比較trigram和bigram的差異集合

In [1]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from enum import Enum
from pprint import pprint
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from gensim.models import Phrases

import pyLDAvis
import pyLDAvis.gensim
from gensim.models.ldamodel import LdaModel

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.DEBUG)

2019-11-04 23:15:26,648 : DEBUG : Loaded backend module://ipykernel.pylab.backend_inline version unknown.


## 自定義 data types and functions

In [2]:
class ContentType(Enum):
    TIT = 'title'
    ABS = 'abstract'
    AUT = 'author'
    SEC = 'section'
    
def get_contents(content_type):
    all_contents = []
    dataset_path = '../dataset'
    for file in os.listdir(dataset_path):
        file_path = os.path.join(dataset_path, file)
        if os.path.isfile(file_path):
            with open(file_path) as f:  
                line = f.readlines()
                if content_type == ContentType.AUT:
                    line = line[1]
                elif content_type == ContentType.SEC:
                    line = line[2]
                elif content_type == ContentType.ABS:
                    line = line[3]
                else:
                    line = line[0]
                line = line.strip()
                all_contents.append(line)
        else:
            print(file_path + ' does not exist.')
    return all_contents


def get_all_titles():
    return get_contents(ContentType.TIT)

def get_all_authors():        
    return get_contents(ContentType.AUT)

def get_all_sections():
    return get_contents(ContentType.SEC)

def get_all_abstracts():
    return get_contents(ContentType.ABS)

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            token = wordnet_lemmatizer.lemmatize(token, pos='v')
            token = wordnet_lemmatizer.lemmatize(token, pos='n')
            result.append(token)
    return result

def get_chart_data(num_topics,num_words,topics):
    buff = 300
    K = num_topics
    topicWordProbMat = topics
    
    #columns = ['1','2','3','4','5']
    columns = range(1,num_topics+1)

    df = pd.DataFrame(columns = columns)
    pd.set_option('display.width', 1000)

    # 40 will be resized later to match number of words in DC
    zz = np.zeros(shape=(buff,K))

    last_number = 0
    DC = {}

    for x in range (num_words): #取每個topic前10個字
        data= pd.DataFrame(columns=columns,index=[0])
        for i in range(num_topics):
            data[columns[i]] = ""
        df = df.append(data,ignore_index=True)  

    for line in topicWordProbMat:
        topic_id,words = line #一個line是一個topic
        probs = words.split("+")
        y = 0 #用來算第幾個word
        for pr in probs:    
            a = pr.split("*")
            df.iloc[y,topic_id] = a[1] #該word

            if a[1] in DC:
                zz[DC[a[1]]][topic_id] = a[0] #該word的機率
            else:
                zz[last_number][topic_id] = a[0]
                DC[a[1]] = last_number
                last_number = last_number+1
            y = y + 1

    return (df,DC,zz)

def show_words_table(df):  
    print(df)
    print('\n')
    
def show_dictionary(DC):
    print(DC)
    print('字典字數：',len(DC))
    print('\n')

def show_probs_table(zz):
    print(zz)
    print(zz.shape)
    
def show_heapmap(DC,zz):
    %matplotlib inline

    zz = np.resize(zz,(len(DC.keys()),zz.shape[1]))

    for val, key in enumerate(DC.keys()):
            plt.text(-3.5, val + 0.1, key,
                     horizontalalignment='right',
                     verticalalignment='center'
                     )

    #plt.figure(figsize=(10,50))
    plt.imshow(zz, cmap='hot', interpolation='nearest',aspect=0.5)#'auto'
    plt.show()

### 取出所有摘要

In [3]:
contents = get_all_abstracts()
print('共',len(contents),'篇論文\n')

documents = pd.DataFrame(data=contents,columns=['abstract'])
documents['index'] = documents.index
documents[:10]

共 1343 篇論文



abstract  index
0  We consider the problem of actively eliciting ...      0
1  We investigate the task of distractor generati...      1
2  The most common representation formalisms for ...      2
3  Statistical relational learning models are pow...      3
4  Multimodal representation learning is gaining ...      4
5  Reinforcement learning (RL) has shown its adva...      5
6  Selecting appropriate tutoring help actions th...      6
7  Recognizing time expressions is a fundamental ...      7
8  When facing large-scale image datasets, online...      8
9  Temporal modeling in videos is a fundamental y...      9

### 預處理的全部論文摘要

In [4]:
processed_docs = documents['abstract'].map(preprocess)

In [ ]:
# bigram

bigram = Phrases(processed_docs,min_count=10)

for idx in range(len(processed_docs)):
    for token in bigram[processed_docs[idx]]:
        if '_' in token:
            processed_docs[idx].append(token)

In [5]:
# bigram and trigram

bigram = Phrases(processed_docs, min_count=10)
trigram = Phrases(bigram[processed_docs], min_count=10)

for i in range(len(processed_docs)):
    for token in bigram[processed_docs[i]]:
        if token.count('_') == 1:
            processed_docs[i].append(token)    
    for token in trigram[bigram[processed_docs[i]]]:
        if token.count('_') == 2:
            processed_docs[i].append(token)

2019-11-04 23:15:36,535 : INFO : collecting all words and their counts
2019-11-04 23:15:36,538 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-11-04 23:15:36,770 : INFO : collected 89298 word types from a corpus of 121739 words (unigram + bigrams) and 1343 sentences
2019-11-04 23:15:36,771 : INFO : using 89298 counts as vocab in Phrases<0 vocab, min_count=10, threshold=10.0, max_vocab_size=40000000>
2019-11-04 23:15:36,772 : INFO : collecting all words and their counts
2019-11-04 23:15:36,773 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-11-04 23:15:37,356 : INFO : collected 91908 word types from a corpus of 115388 words (unigram + bigrams) and 1343 sentences
2019-11-04 23:15:37,357 : INFO : using 91908 counts as vocab in Phrases<0 vocab, min_count=10, threshold=10.0, max_vocab_size=40000000>


## Dataset

### 產生字典

In [11]:
dictionary = gensim.corpora.Dictionary(processed_docs)
print('共',len(dictionary),'個字\n')

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

dictionary.save('../corpus/dict_trigram_filtered.dict')
#loaded_dict = corpora.Dictionary.load('../corpus/mydict.dict')

2019-11-04 23:25:16,146 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-11-04 23:25:16,327 : INFO : built Dictionary(7170 unique tokens: ['active', 'active_learn', 'actively', 'adaptive', 'aggregation']...) from 1343 documents (total 128849 corpus positions)
2019-11-04 23:25:16,339 : INFO : discarding 5584 tokens: [('actively', 9), ('algorithm', 358), ('approach', 483), ('coefficient', 8), ('collective', 7), ('elicitation', 3), ('equity', 2), ('gini', 1), ('interleave', 3), ('knapsack', 5)]...
2019-11-04 23:25:16,340 : INFO : keeping 1586 tokens which were in no less than 10 and no more than 268 (=20.0%) documents
2019-11-04 23:25:16,342 : DEBUG : rebuilding dictionary, shrinking gaps
2019-11-04 23:25:16,346 : INFO : resulting dictionary: Dictionary(1586 unique tokens: ['active', 'active_learn', 'adaptive', 'aggregation', 'allow']...)
2019-11-04 23:25:16,348 : INFO : saving Dictionary object under ../corpus/dict_trigram_filtered.dict, separately None
2019-11-04 23:2

共 7170 個字

Number of unique words after removing rare and common words: 1586


In [12]:
dic1 = [] 
for v in dictionary.itervalues():
    if v.count('_') == 2:
        dic1.append(v)
print('共有',len(dic1),'個字')
pprint(dic1)

共有 28 個字
['markov_decision_process',
 'extensive_experiment_conduct',
 'paper_propose_novel',
 'semi_supervise_learn',
 'generative_adversarial_network',
 'play_important_role',
 'recurrent_neural_network',
 'achieve_state_result',
 'deep_neural_network',
 'paper_present_novel',
 'experimental_result_demonstrate',
 'natural_language_process',
 'convolutional_neural_network',
 'outperform_state_method',
 'multi_task_learn',
 'experiment_real_world',
 'demonstrate_effectiveness_propose',
 'extensive_experimental_result',
 'graph_convolutional_network',
 'deep_reinforcement_learn',
 'conduct_extensive_experiment',
 'neural_machine_translation',
 'achieve_state_performance',
 'unsupervised_domain_adaptation',
 'real_world_datasets',
 'significantly_outperform_state',
 'real_world_application',
 'significantly_improve_performance']


### 產生 bag of words corpus

In [10]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print('共',len(bow_corpus),'筆')

corpora.MmCorpus.serialize('../corpus/corpus_trigram.mm', bow_corpus)
#corpus_test = corpora.MmCorpus('../corpus/bow_corpus.mm')

2019-11-04 23:23:40,550 : INFO : storing corpus in Matrix Market format to ../corpus/corpus_trigram.mm
2019-11-04 23:23:40,550 : WARNING : this function is deprecated, use smart_open.open instead
/Users/alexis/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:160: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  args, varargs, keywords, defaults = inspect.getargspec(kallable)
2019-11-04 23:23:40,551 : DEBUG : {'uri': '../corpus/corpus_trigram.mm', 'mode': 'wb+', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'transport_params': {}}
2019-11-04 23:23:40,553 : INFO : saving sparse matrix to ../corpus/corpus_trigram.mm
2019-11-04 23:23:40,556 : INFO : PROGRESS: saving document #0
2019-11-04 23:23:40,666 : INFO : PROGRESS: saving document #1000
2019-11-04 23:23:40,698 : INFO : saved 1343x7170 matrix, density=0.943% (90810/9629310)
2019-11-0

共 1343 筆


In [ ]:
# 不要執行
# 奇怪的現象 (被吃了三個字)

dic1 = [] # uni-gram
for v in dictionary.itervalues():
    dic1.append(v)
print('uni-gram dictionary 共有',len(dic1),'個字')

dic2 = [] #bi-gram
for v in dictionary2.itervalues():
    dic2.append(v)
print('bi-gram dictionary 共有',len(dic2),'個字')

print('\n')

print('uni-gram dic 比 bi-gram dictionary 多了',len(list(set(dic1)-set(dic2))),'個字')
print(list(set(dic1) - set(dic2)))

print('\n')

print('bi-gram dic 比 uni-gram dictionary 多了',len(list(set(dic2)-set(dic1))),'個字')
print(list(set(dic2) - set(dic1)))